In [1]:
# Run dependencies
%run extraction.ipynb

In [2]:
import numpy as np
import nest_asyncio
nest_asyncio.apply()

# Transformation

### (1) Walmart 2010-2012 Sales Dataset/API Holiday

In [3]:
## *******************************************TRANSFORMATION*******************************************
## (1) WALMART 2010-2012 SALES DATASET/API HOLIDAY

# Merge stores and feature on Store
features_stores = features.merge(stores, how='inner', on='Store')

# Convert date column from string to date type
features_stores.Date = pd.to_datetime(features_stores.Date)
train.Date = pd.to_datetime(train.Date)

# Add column week and year
features_stores['Week'] = features_stores.Date.dt.week 
features_stores['Year'] = features_stores.Date.dt.year

# Merge datasets
walmart_data = train.merge(features_stores, 
                           how='inner',
                           on=['Store','Date','IsHoliday']).sort_values(by=['Store',
                                                                            'Dept',
                                                                            'Date']).reset_index(drop=True)

# Drop MarkDown columns and IsHoliday
walmart_data = walmart_data.drop(["IsHoliday", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"], axis=1)

# Drop NaN values in CPI and Unemployment
walmart_data = walmart_data.dropna()

# Change Temprature column name to Temperature F
walmart_data=walmart_data.rename(columns={"Temperature": "Temperature F"})

# Convert temprature from Fahrenheit to Celsius
temp_celcius=[]
for temperature in walmart_data['Temperature F']:
    celcius = (temperature - 32) * (5.0/9.0)
    temp_celcius.append(celcius)
walmart_data["Temperature C"] = temp_celcius

# Rounding Column values to required decimal points
walmart_data = walmart_data.round({'Fuel_Price': 2, 'Temperature C': 0, 'CPI': 4 })

# Convert Date to datetime and subtract 6 days to get Start of Week
walmart_data['Start of Week'] = pd.to_datetime(walmart_data.Date) - timedelta(days=6)

# Sort dataframe and reset index for walmart
walmart_data = walmart_data.sort_values(by =['Date']).reset_index(drop = True)

In [4]:
## (1) WALMART 2010-2012 SALES DATASET/API HOLIDAY

# Sort dataframe and reset index for walmart
holiday_df = holiday_df.sort_values(by =['Date']).reset_index(drop = True)

# Convert holiday date
holiday_df['Date'] = pd.to_datetime(holiday_df['Date'])

holiday_name = []
k = 0
# Label data with Holiday Name or No Holiday
for i in range(len(walmart_data)):
    
    if (holiday_df['Date'][k] <= walmart_data['Date'][i]) & (holiday_df['Date'][k] >= walmart_data['Start of Week'][i]):
        holiday_name.append(holiday_df['Holiday'][k])
    
    elif (holiday_df['Date'][k] > walmart_data['Date'][i]):
        holiday_name.append("No Holiday")
    
    elif (walmart_data['Date'][i] > holiday_df['Date'][k]):
        holiday_name.append("No Holiday")
        k +=1

# Set new column as list created:
walmart_data['Holiday Name'] = holiday_name

# Convert Christmas Day and Eve to be Christmas
walmart_data['Holiday Name'] = walmart_data['Holiday Name'].replace({'Christmas Day': 'Christmas',
                                                                    'Christmas Eve': 'Christmas'})

# Display preview of dataframe:
walmart_data.head()

Store  Dept       Date  Weekly_Sales  Temperature F  Fuel_Price       CPI  \
0      1     1 2010-02-05      24924.50          42.31        2.57  211.0964   
1     29     5 2010-02-05      15552.08          24.36        2.79  131.5279   
2     29     6 2010-02-05       3200.22          24.36        2.79  131.5279   
3     29     7 2010-02-05      10820.05          24.36        2.79  131.5279   
4     29     8 2010-02-05      20055.64          24.36        2.79  131.5279   

   Unemployment Type    Size  Week  Year  Temperature C Start of Week  \
0         8.106    A  151315     5  2010            6.0    2010-01-30   
1        10.064    B   93638     5  2010           -4.0    2010-01-30   
2        10.064    B   93638     5  2010           -4.0    2010-01-30   
3        10.064    B   93638     5  2010           -4.0    2010-01-30   
4        10.064    B   93638     5  2010           -4.0    2010-01-30   

  Holiday Name  
0   No Holiday  
1   No Holiday  
2   No Holiday  
3   No Holiday  
4   No Holiday

In [5]:
## (1) WALMART 2010-2012 SALES DATASET/API HOLIDAY
## TABLE 1 SUMMARY WALMART STORES SALES OVER 2010-2012

# Aggregate over Store/Start of Week, Include Other Columns
header = ['Store','Start of Week', 'Date','Fuel_Price','CPI','Unemployment','Type', 'Size','Week','Year','Temperature C','Holiday Name']
walmart = pd.DataFrame(walmart_data.groupby(header)['Weekly_Sales'].sum())

# Reset Index and add ID Column
walmart = walmart.reset_index()
walmart = walmart.reset_index(drop = False)

# Rename Columns
walmart.columns = ['ID','Store','Start_of_Week','Week_Date','Fuel_Price','CPI','Unemployment','Type','Size','Week','Year',
                  'Temperature_C','Holiday_Name','Weekly_Sales']

# Display preview
walmart.head()

ID  Store Start_of_Week  Week_Date  Fuel_Price       CPI  Unemployment  \
0   0      1    2010-01-30 2010-02-05        2.57  211.0964         8.106   
1   1      1    2010-02-06 2010-02-12        2.55  211.2422         8.106   
2   2      1    2010-02-13 2010-02-19        2.51  211.2891         8.106   
3   3      1    2010-02-20 2010-02-26        2.56  211.3196         8.106   
4   4      1    2010-02-27 2010-03-05        2.62  211.3501         8.106   

  Type    Size  Week  Year  Temperature_C Holiday_Name  Weekly_Sales  
0    A  151315     5  2010            6.0   No Holiday    1643690.90  
1    A  151315     6  2010            4.0   No Holiday    1641957.44  
2    A  151315     7  2010            4.0   No Holiday    1611968.17  
3    A  151315     8  2010            8.0   No Holiday    1409727.59  
4    A  151315     9  2010            8.0   No Holiday    1554806.68

### (2) API HOLIDAY

In [6]:
## (2) API HOLIDAY
## TABLE 2 HOLIDAY DATE LIST 2010-2012

# Display preview
holiday_df.head()

Holiday  Year       Date
0    New Year's Day  2010 2010-01-01
1  Independence Day  2010 2010-07-04
2  Thanksgiving Day  2010 2010-11-25
3     Christmas Eve  2010 2010-12-24
4     Christmas Day  2010 2010-12-25

### (3) Walmart Stock 1972-2020 Dataset

In [7]:
## (3) STOCK 1972-2020 DATASET
## TABLE 3 ALL 2010-2012 STOCK DAILY DATASET

# Convert Date Columns to datetime
stock['Date'] = pd.to_datetime(stock['Date'])
stock['Year'] = stock['Date'].dt.year

# Set first date and last date on sales dateframe
first_date = walmart['Start_of_Week'][0]
last_date = walmart['Week_Date'].iloc[-1]

# Filter stock date with sales data range
stock = stock.loc[(stock['Date'] >= first_date) & (stock['Date'] <=last_date)]
stock = stock.reset_index(drop = True)

# Sort by Date
stock = stock.sort_values(by = 'Date')
walmart = walmart.sort_values(by = 'Week_Date')

# Get list of unique walmart weekly dates
walmart_date = walmart.Week_Date.unique()

# Classify Date under weekly date
wk_date = []
w = 0

for s in range(len(stock)):
    if stock['Date'][s] <= walmart_date[w]:
        wk_date.append(walmart_date[w])
    else:
        wk_date.append(walmart_date[w+1])
        w = w + 1

# Add Week Date and rename columns
stock['Week_Date'] = wk_date
stock.columns = ['Date','Open','High','Low','Close','Adj_Close','Volume','Year','Week_Date']

# Display preview
stock.head()

Date       Open       High        Low      Close  Adj_Close    Volume  \
0 2010-02-01  53.619999  53.779999  53.310001  53.480000  40.977615  11019500   
1 2010-02-02  53.590000  53.720001  53.330002  53.490002  40.985275  11387900   
2 2010-02-03  53.730000  54.500000  53.639999  54.270000  41.582928  17988900   
3 2010-02-04  53.880001  54.299999  52.959999  52.970001  40.586830  21029900   
4 2010-02-05  52.770000  53.529999  52.759998  53.450001  40.954620  15545800   

   Year  Week_Date  
0  2010 2010-02-05  
1  2010 2010-02-05  
2  2010 2010-02-05  
3  2010 2010-02-05  
4  2010 2010-02-05

### (5) Walmart Store Listing Json

In [8]:
## (5) WALMART STORE JSON
## TABLE _ ALL WALMART STORE LISTING

# Filter for Walmart Supercenter 
store = store[store.storeType.eq(1)]

# Display preview
store = store[['id', 'address1','city','state','latitude','longitude']]

In [9]:
# Replace state abbreviation with state integer ID
state_id = state_id.rename(columns = {'index': 'state_id','STATE':'state'})
store = store.merge(state_id, how='inner',on='state')
store = store.drop('state',1)
store.head()

id                     address1         city   latitude   longitude  \
0  2074  1350 S Seward Meridian Pkwy      Wasilla  61.568752 -149.363647   
1  2070                    3101 A St    Anchorage  61.192337 -149.880356   
2  2071          8900 Old Seward Hwy    Anchorage  61.140263 -149.868835   
3  2188         18600 Eagle River Rd  Eagle River  61.309483 -149.534912   
4  3814             6525 Glacier Hwy       Juneau  58.358349 -134.514862   

  state_id  
0       02  
1       02  
2       02  
3       02  
4       02

### (6) Walmart Marketshare

In [10]:
#Remove '%' from 'MARKET_SHARE', then view
marketShare_df['MARKET_SHARE'] = marketShare_df['MARKET_SHARE'].str.replace('%','')

#Convert from object ('MARKET_SHARE') to int
marketShare_df['MARKET_SHARE'] = marketShare_df['MARKET_SHARE'].astype(int)
marketShare_df['STATE'] = marketShare_df['STATE'].replace(abbr_id, regex=True)
marketShare_final = marketShare_df.rename(columns={"STATE":"STATE_ID"})
marketShare_final.head()

CITY STATE_ID  POPULATION  MARKET_SHARE
0  Atchison       20       16580            95
1  Portales       35       19730            95
2  Sterling       08       22068            91
3    Deming       35       24699            90
4    Guymon       39       21385            90

### (4) Walmart Sales Dataset (5 Years) ZIP File

In [11]:
## (4) WALMART SALES DATA (5 YEARS) ZIP FILE SALES_AUG & PRICES CSV CONT.

# Due to data size take 10% random sample of sales_aug and prices
sales_aug = sales_aug.sample(4300)
id_sample = sales_aug['id']
prices = prices[prices['id'].isin(id_sample)]
sales_aug = sales_aug.reset_index(drop=True); prices = prices.reset_index(drop=True)
df_prices_raw = prices
df_sales_raw = sales_aug

In [12]:
df_prices_raw.head()

id   d_1   d_2   d_3   d_4   d_5   d_6   d_7   d_8   d_9  \
0  FOODS_1_001_CA_2  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
1  FOODS_1_001_CA_3  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
2  FOODS_1_001_CA_4  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
3  FOODS_1_001_WI_2  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
4  FOODS_1_002_CA_1  7.88  7.88  7.88  7.88  7.88  7.88  7.88  7.88  7.88   

   ...  d_1960  d_1961  d_1962  d_1963  d_1964  d_1965  d_1966  d_1967  \
0  ...    2.24    2.24    2.24    2.24    2.24    2.24    2.24    2.24   
1  ...    2.24    2.24    2.24    2.24    2.24    2.24    2.24    2.24   
2  ...    2.24    2.24    2.24    2.24    2.24    2.24    2.24    2.24   
3  ...    2.24    2.24    2.24    2.24    2.24    2.24    2.24    2.24   
4  ...    9.48    9.48    9.48    9.48    9.48    9.48    9.48    9.48   

   d_1968  d_1969  
0    2.24    2.24  
1    2.24    2.24  
2    2.24    2.24  
3    2.24    2.24  
4    9.48    9.48  

[5 rows x 1970 columns]

In [13]:
# the sales data has subtotals that should be dropped
df_sales_raw.drop(df_sales_raw[df_sales_raw['item_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['state_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['store_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['cat_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['dept_id'] == 'X'].index, inplace=True)

df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'WI'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'TX'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'CA'].index, inplace=True)

df_sales_raw.head()

id          item_id      dept_id     cat_id store_id  \
0      FOODS_3_429_TX_3      FOODS_3_429      FOODS_3      FOODS     TX_3   
1      FOODS_1_001_CA_3      FOODS_1_001      FOODS_1      FOODS     CA_3   
4      FOODS_3_001_TX_2      FOODS_3_001      FOODS_3      FOODS     TX_2   
5  HOUSEHOLD_1_026_WI_2  HOUSEHOLD_1_026  HOUSEHOLD_1  HOUSEHOLD     WI_2   
6      FOODS_3_514_TX_2      FOODS_3_514      FOODS_3      FOODS     TX_2   

  state_id  d_1  d_2  d_3  d_4  ...  d_1937  d_1938  d_1939  d_1940  d_1941  \
0       TX    0    0    0    0  ...       2       0       0       1       1   
1       CA    1    2    1    1  ...       1       0       3       2       2   
4       TX    0    0    0    2  ...       0       0       1       4       0   
5       WI    0    0    0    0  ...       0       8       1       3       1   
6       TX    0    0    0    0  ...       0       4       1       1       5   

     sales1    sales2  start    scale1    scale2  
0  0.000014  0.000024   1073  0.978546  0.990773  
1  0.000013  0.000014      1  1.607012  1.601341  
4  0.000001  0.000001      4  0.542977  0.541322  
5  0.000022  0.000028   1125  1.238882  1.250307  
6  0.000001  0.000003    986  0.326134  0.349057  

[5 rows x 1952 columns]

In [14]:
# create category data from the prices/sales/quant data
df_sales_id = df_sales_raw.groupby('id').size().reset_index()
df_sales_id = df_sales_id.loc[:, ['id']]
df_sales_id.rename(columns = {'id': 'id_desc'}, inplace=True) 
df_sales_id['id'] = df_sales_id.index

df_sales_categories = df_sales_raw[['cat_id']].dropna(axis=0).groupby('cat_id').size().reset_index()
df_sales_categories = df_sales_categories.loc[:, ['cat_id']]
df_sales_categories.rename(columns = {'cat_id': 'cat_desc'}, inplace=True) 
df_sales_categories['cat_id'] = df_sales_categories.index

df_sales_departments = df_sales_raw[['dept_id']].dropna(axis=0).groupby('dept_id').size().reset_index()
df_sales_departments = df_sales_departments.loc[:, ['dept_id']]
df_sales_departments.rename(columns = {'dept_id': 'dept_desc'}, inplace=True) 
df_sales_departments['dept_id'] = df_sales_departments.index

df_sales_stores = df_sales_raw[['store_id']].dropna(axis=0).groupby('store_id').size().reset_index()
df_sales_stores = df_sales_stores.loc[:, ['store_id']]
df_sales_stores.rename(columns = {'store_id': 'store_desc'}, inplace=True) 
df_sales_stores['store_id'] = df_sales_stores.index 

In [15]:
# manually create the states dataframe - to align the keys with other tables being used!
states = [[6,'CA'],[48,'TX'],[55,'WI']]
df_sales_states = pd.DataFrame(states, columns = ['state_id', 'state_desc'])

df_sales_items = df_sales_raw[['item_id']].dropna(axis=0).groupby('item_id').size().reset_index()
df_sales_items = df_sales_items.loc[:, ['item_id']]
df_sales_items.rename(columns = {'item_id': 'item_desc'}, inplace=True) 
df_sales_items['item_id'] = df_sales_items.index

In [16]:
# add the categories to prices 
# since we're removing the daily sales data with 0 sales, we need to have all the id for the prices table
# so the table can be further summarized
df_prices_raw = df_prices_raw.merge(df_sales_raw[['id','dept_id','cat_id','item_id','store_id','state_id']], how='left', left_on='id', right_on='id')
df_prices_raw


id    d_1    d_2    d_3    d_4    d_5    d_6    d_7  \
0         FOODS_1_001_CA_2   2.00   2.00   2.00   2.00   2.00   2.00   2.00   
1         FOODS_1_001_CA_3   2.00   2.00   2.00   2.00   2.00   2.00   2.00   
2         FOODS_1_001_CA_4   2.00   2.00   2.00   2.00   2.00   2.00   2.00   
3         FOODS_1_001_WI_2   2.00   2.00   2.00   2.00   2.00   2.00   2.00   
4         FOODS_1_002_CA_1   7.88   7.88   7.88   7.88   7.88   7.88   7.88   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
3033  HOUSEHOLD_2_509_TX_2   8.97   8.97   8.97   8.97   8.97   8.97   8.97   
3034  HOUSEHOLD_2_509_WI_3   8.97   8.97   8.97   8.97   8.97   8.97   8.97   
3035  HOUSEHOLD_2_511_WI_1   2.87   2.87   2.87   2.87   2.87   2.87   2.87   
3036  HOUSEHOLD_2_513_WI_3    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3037  HOUSEHOLD_2_514_CA_3  18.97  18.97  18.97  18.97  18.97  18.97  18.97   

        d_8    d_9  ...  d_1965  d_1966  d_1967  d_1968  d_1969      dept_id  \
0      2.00   2.00  ...    2.24    2.24    2.24    2.24    2.24      FOODS_1   
1      2.00   2.00  ...    2.24    2.24    2.24    2.24    2.24      FOODS_1   
2      2.00   2.00  ...    2.24    2.24    2.24    2.24    2.24      FOODS_1   
3      2.00   2.00  ...    2.24    2.24    2.24    2.24    2.24      FOODS_1   
4      7.88   7.88  ...    9.48    9.48    9.48    9.48    9.48      FOODS_1   
...     ...    ...  ...     ...     ...     ...     ...     ...          ...   
3033   8.97   8.97  ...    8.97    8.97    8.97    8.97    8.97  HOUSEHOLD_2   
3034   8.97   8.97  ...    8.97    8.97    8.97    8.97    8.97  HOUSEHOLD_2   
3035   2.87   2.87  ...    2.74    2.74    2.74    2.74    2.74  HOUSEHOLD_2   
3036    NaN    NaN  ...    2.78    2.78    2.78    2.78    2.78  HOUSEHOLD_2   
3037  18.97  18.97  ...   19.54   19.54   19.54   19.54   19.54  HOUSEHOLD_2   

         cat_id          item_id  store_id  state_id  
0         FOODS      FOODS_1_001      CA_2        CA  
1         FOODS      FOODS_1_001      CA_3        CA  
2         FOODS      FOODS_1_001      CA_4        CA  
3         FOODS      FOODS_1_001      WI_2        WI  
4         FOODS      FOODS_1_002      CA_1        CA  
...         ...              ...       ...       ...  
3033  HOUSEHOLD  HOUSEHOLD_2_509      TX_2        TX  
3034  HOUSEHOLD  HOUSEHOLD_2_509      WI_3        WI  
3035  HOUSEHOLD  HOUSEHOLD_2_511      WI_1        WI  
3036  HOUSEHOLD  HOUSEHOLD_2_513      WI_3        WI  
3037  HOUSEHOLD  HOUSEHOLD_2_514      CA_3        CA  

[3038 rows x 1975 columns]

In [17]:
# add the foreign keys for categories, departments, items, states and stores to the ***prices*** table
# rename the previous non-numeric ids (delete them after)
df_prices_raw.rename(columns = {'id': 'id_old','cat_id': 'cat_id_old','dept_id': 'dept_id_old','item_id': 'item_id_old','state_id': 'state_id_old','store_id': 'store_id_old'}, inplace=True)

# merge df_prices (left) with df_sales_id on id_old = id_desc to get the new numeric id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_id, how='left', left_on='id_old', right_on='id_desc')
df_prices_raw['id'] = df_prices_raw['id'].astype('Int64')

# merge df_prices (left) with df_sales_categories on cat_id_old = cat_id_desc to get the new numeric cat_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_categories, how='left', left_on='cat_id_old', right_on='cat_desc')
df_prices_raw['cat_id'] = df_prices_raw['cat_id'].astype('Int64')

# merge df_prices (left) with df_sales_departments on dept_id_old = dept_id_desc to get the new numeric dept_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_departments, how='left', left_on='dept_id_old', right_on='dept_desc')
df_prices_raw['dept_id'] = df_prices_raw['dept_id'].astype('Int64')

# merge df_prices (left) with df_sales_items on item_id_old = item_id_desc to get the new numeric item_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_items, how='left', left_on='item_id_old', right_on='item_desc')
df_prices_raw['item_id'] = df_prices_raw['item_id'].astype('Int64')

# merge df_prices (left) with df_sales_states on state_id_old = state_id_desc to get the new numeric state_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_states, how='left', left_on='state_id_old', right_on='state_desc')
df_prices_raw['state_id'] = df_prices_raw['state_id'].astype('Int64')

# merge df_prices (left) with df_sales_stores on store_id_old = store_id_desc to get the new numeric store_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_stores, how='left', left_on='store_id_old', right_on='store_desc')
df_prices_raw['store_id'] = df_prices_raw['store_id'].astype('Int64')

df_prices_raw.head()

id_old   d_1   d_2   d_3   d_4   d_5   d_6   d_7   d_8   d_9  \
0  FOODS_1_001_CA_2  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
1  FOODS_1_001_CA_3  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
2  FOODS_1_001_CA_4  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
3  FOODS_1_001_WI_2  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00   
4  FOODS_1_002_CA_1  7.88  7.88  7.88  7.88  7.88  7.88  7.88  7.88  7.88   

   ...  cat_desc  cat_id  dept_desc  dept_id    item_desc  item_id  state_id  \
0  ...     FOODS       0    FOODS_1        0  FOODS_1_001        0         6   
1  ...     FOODS       0    FOODS_1        0  FOODS_1_001        0         6   
2  ...     FOODS       0    FOODS_1        0  FOODS_1_001        0         6   
3  ...     FOODS       0    FOODS_1        0  FOODS_1_001        0        55   
4  ...     FOODS       0    FOODS_1        0  FOODS_1_002        1         6   

   state_desc  store_desc  store_id  
0          CA        CA_2         1  
1          CA        CA_3         2  
2          CA        CA_4         3  
3          WI        WI_2         8  
4          CA        CA_1         0  

[5 rows x 1987 columns]

In [18]:
# add the foreign keys for categories, departments, items, states and stores to the ***sales*** table
# rename the previous non-numeric ids (delete them after)
df_sales_raw.rename(columns = {'id': 'id_old','cat_id': 'cat_id_old','dept_id': 'dept_id_old','item_id': 'item_id_old', 'state_id': 'state_id_old','store_id': 'store_id_old'}, inplace=True)

# merge df_sales (left) with df_sales_id on id_old = id_desc to get the new numeric id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_id, how='left', left_on='id_old', right_on='id_desc')
df_sales_raw['id'] = df_sales_raw['id'].astype('Int64')

# merge df_sales (left) with df_sales_categories on cat_id_old = cat_id_desc to get the new numeric cat_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_categories, how='left', left_on='cat_id_old', right_on='cat_desc')
df_sales_raw['cat_id'] = df_sales_raw['cat_id'].astype('Int64')

# merge df_sales (left) with df_sales_departments on dept_id_old = dept_id_desc to get the new numeric dept_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_departments, how='left', left_on='dept_id_old', right_on='dept_desc')
df_sales_raw['dept_id'] = df_sales_raw['dept_id'].astype('Int64')

# merge df_sales (left) with df_sales_items on item_id_old = item_id_desc to get the new numeric item_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_items, how='left', left_on='item_id_old', right_on='item_desc')
df_sales_raw['item_id'] = df_sales_raw['item_id'].astype('Int64')

# merge df_sales (left) with df_sales_states on state_id_old = state_id_desc to get the new numeric state_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_states, how='left', left_on='state_id_old', right_on='state_desc')
df_sales_raw['state_id'] = df_sales_raw['state_id'].astype('Int64')

# merge df_sales (left) with df_sales_stores on store_id_old = store_id_desc to get the new numeric store_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_stores, how='left', left_on='store_id_old', right_on='store_desc')
df_sales_raw['store_id'] = df_sales_raw['store_id'].astype('Int64')

df_sales_raw.head()

id_old      item_id_old  dept_id_old cat_id_old store_id_old  \
0      FOODS_3_429_TX_3      FOODS_3_429      FOODS_3      FOODS         TX_3   
1      FOODS_1_001_CA_3      FOODS_1_001      FOODS_1      FOODS         CA_3   
2      FOODS_3_001_TX_2      FOODS_3_001      FOODS_3      FOODS         TX_2   
3  HOUSEHOLD_1_026_WI_2  HOUSEHOLD_1_026  HOUSEHOLD_1  HOUSEHOLD         WI_2   
4      FOODS_3_514_TX_2      FOODS_3_514      FOODS_3      FOODS         TX_2   

  state_id_old  d_1  d_2  d_3  d_4  ...   cat_desc  cat_id    dept_desc  \
0           TX    0    0    0    0  ...      FOODS       0      FOODS_3   
1           CA    1    2    1    1  ...      FOODS       0      FOODS_1   
2           TX    0    0    0    2  ...      FOODS       0      FOODS_3   
3           WI    0    0    0    0  ...  HOUSEHOLD       2  HOUSEHOLD_1   
4           TX    0    0    0    0  ...      FOODS       0      FOODS_3   

   dept_id        item_desc  item_id  state_id  state_desc  store_desc  \
0        2      FOODS_3_429      697        48          TX        TX_3   
1        0      FOODS_1_001        0         6          CA        CA_3   
2        2      FOODS_3_001      411        48          TX        TX_2   
3        5  HOUSEHOLD_1_026     1336        55          WI        WI_2   
4        2      FOODS_3_514      748        48          TX        TX_2   

   store_id  
0         6  
1         2  
2         5  
3         8  
4         5  

[5 rows x 1964 columns]

In [19]:
# drop old non-numeric ids and redundant merged descriptions
df_sales_raw.drop(['id_old','id_desc','cat_id_old','cat_desc','dept_id_old','dept_desc','item_id_old','item_desc','state_id_old','state_desc','store_id_old','store_desc'],axis=1,inplace=True)
df_sales_raw

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  ...    sales2  start  \
0       0    0    0    0    0    0    0    0    0     0  ...  0.000024   1073   
1       1    2    1    1    1    2    0    1    1     1  ...  0.000014      1   
2       0    0    0    2    2    0    1    2    1     0  ...  0.000001      4   
3       0    0    0    0    0    0    0    0    0     0  ...  0.000028   1125   
4       0    0    0    0    0    0    0    0    0     0  ...  0.000003    986   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...       ...    ...   
3033    0    0    0    0    0    0    0    0    0     0  ...  0.000002   1003   
3034    0    0    0    0    0    0    0    0    0     0  ...  0.000002    173   
3035    1    0    0    1    1    0    1    1    0     0  ...  0.000013      1   
3036    2    2    0    2    1    1    0    3    1     0  ...  0.000002      1   
3037    0    0    0    0    0    0    0    0    0     0  ...  0.000003    461   

        scale1    scale2    id  cat_id  dept_id  item_id  state_id  store_id  
0     0.978546  0.990773  1047       0        2      697        48         6  
1     1.607012  1.601341     1       0        0        0         6         2  
2     0.542977  0.541322   630       0        2      411        48         5  
3     1.238882  1.250307  2022       2        5     1336        55         8  
4     0.326134  0.349057  1129       0        2      748        48         5  
...        ...       ...   ...     ...      ...      ...       ...       ...  
3033  0.795380  0.789755  2407       2        5     1585        55         7  
3034  0.484186  0.481041  2054       2        5     1354        55         8  
3035  0.694401  0.708613  1274       0        2      835        48         4  
3036  0.721088  0.718927   953       0        2      635        48         5  
3037  0.352860  0.352265  1883       1        4     1239        55         7  

[3038 rows x 1952 columns]

In [20]:
# the start and scale columns shouldn't be transposed in this table because they're item and store level
# start is not needed because the 0 sales roles will be deleted after being transposed
# the scale should be recalculated if needed on the grouped amounts

df_sales_raw.drop(['sales1','sales2','start','scale1','scale2'],axis=1,inplace=True)
df_sales_raw.head()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  ...  d_1938  d_1939  \
0    0    0    0    0    0    0    0    0    0     0  ...       0       0   
1    1    2    1    1    1    2    0    1    1     1  ...       0       3   
2    0    0    0    2    2    0    1    2    1     0  ...       0       1   
3    0    0    0    0    0    0    0    0    0     0  ...       8       1   
4    0    0    0    0    0    0    0    0    0     0  ...       4       1   

   d_1940  d_1941    id  cat_id  dept_id  item_id  state_id  store_id  
0       1       1  1047       0        2      697        48         6  
1       2       2     1       0        0        0         6         2  
2       4       0   630       0        2      411        48         5  
3       3       1  2022       2        5     1336        55         8  
4       1       5  1129       0        2      748        48         5  

[5 rows x 1947 columns]

In [21]:
# Split the Categories
df_ecomm[['category_level1','category_level2','category_level3','category_level4','category_level5','category_level6']] = df_ecomm['Category'].str.split('|',5,expand=True)

# filter out everything that isn't available
df_ecomm.drop(df_ecomm[df_ecomm['Available'] == False].index, inplace=True)

# drop unneeded fields: URL, Name, Description, Brand, Package, Size, Postal Code, GTIN, Crawl Time
df_ecomm.drop(['Uniq Id','Crawl Timestamp','Product Url','Product Name','Description','Item Number','Gtin','Package Size','Postal Code','Category','category_level2','category_level3','category_level4','category_level5','category_level6','Available'],axis=1, inplace=True)

df_ecomm['ecomm_id'] = df_ecomm.index 

df_ecomm.head()

List Price  Sale Price            Brand       category_level1  ecomm_id
0       19.99       19.99    In Style Eyes                Health         0
1       19.99       19.99    In Style Eyes                Health         1
2       19.99       19.99    In Style Eyes                Health         2
3       19.99       19.99    In Style Eyes                Health         3
7       32.51       32.51  CHOSEN SUPPLIES  Household Essentials         7

In [22]:
# transform ***sales*** dataframe (column sales data into rows and add a 'days' field)
for i in range(1, 1942):
    col_name = 'd_' + str(i)
    df_sales_slice = df_sales_raw.loc[:, ['id','item_id','dept_id','cat_id','store_id','state_id',col_name]]
    
    # drop the rows with 0 sales from the slice
    df_sales_slice.drop(df_sales_slice[df_sales_slice[col_name] == 0].index, inplace=True)
    
    # rename the 'd_xxxx' column to sales
    df_sales_slice.rename(columns = {col_name: 'sales'}, inplace = True)
    df_sales_slice['day'] = i
    
    # if it is not the first column, append (i.e. stack) then data; 
    # if it is the first column, create the dataframe  
    if i != 1:
        df_sales = df_sales.append(df_sales_slice, ignore_index=True)
    else:
        df_sales = df_sales_slice
        
    # give a status update every 100 columns processed
    if i % 500 == 0:
        print(str(i) + ' columns processed...')

df_sales.head()

100 columns processed...
200 columns processed...
300 columns processed...
400 columns processed...
500 columns processed...
600 columns processed...
700 columns processed...
800 columns processed...
900 columns processed...
1000 columns processed...
1100 columns processed...
1200 columns processed...
1300 columns processed...
1400 columns processed...
1500 columns processed...
1600 columns processed...
1700 columns processed...
1800 columns processed...
1900 columns processed...


id  item_id  dept_id  cat_id  store_id  state_id  sales  day
0     1        0        0       0         2         6      1    1
1  1076      715        2       0         0         6      4    1
2   987      656        2       0         1         6      5    1
3  2419     1596        5       2         1         6     10    1
4   239      158        1       0         6        48      7    1

In [23]:
# transform ***price*** dataframe (column price data into rows and add a 'days' field) 
for i in range(1, 1970):
    col_name = 'd_' + str(i)
    df_prices_slice = df_prices_raw.loc[:, ['id','item_id','dept_id','cat_id','store_id','state_id',col_name]]
    
    # drop the rows with 0 or null prices from the slice
    df_prices_slice.drop(df_prices_slice[df_prices_slice[col_name] == 0].index, inplace=True)
    df_prices_slice.dropna(inplace=True)
    
    
    df_prices_slice.rename(columns = {col_name: 'price'}, inplace = True) 
    df_prices_slice['day'] = i
    if i != 1:
        df_prices = df_prices.append(df_prices_slice, ignore_index=True)
    else:
        df_prices = df_prices_slice
    
    # give a status update every 100 columns processed
    if i % 500 == 0:
        print(str(i) + ' columns processed...')

df_prices.head()

100 columns processed...
200 columns processed...
300 columns processed...
400 columns processed...
500 columns processed...
600 columns processed...
700 columns processed...
800 columns processed...
900 columns processed...
1000 columns processed...
1100 columns processed...
1200 columns processed...
1300 columns processed...
1400 columns processed...
1500 columns processed...
1600 columns processed...
1700 columns processed...
1800 columns processed...
1900 columns processed...


id  item_id  dept_id  cat_id  store_id  state_id  price  day
0   0        0        0       0         1         6   2.00    1
1   1        0        0       0         2         6   2.00    1
2   2        0        0       0         3         6   2.00    1
3   3        0        0       0         8        55   2.00    1
4   4        1        0       0         0         6   7.88    1

In [24]:
# transform the prices into effective day ranges for any given price by state/store/cat/dept/item
# join the same table to itself on the end date of the range to get the previous price
# create a price increase indicator (0 for down, 1 for up)

# group by on category ids and price to get the min (start) and max (end) date for every price change
df_price_changes_raw = df_prices.groupby(['state_id','store_id','cat_id','dept_id','item_id','price'], level=None).agg({'day':['min','max']}).reset_index()

# flatten the MultiIndex column name
df_price_changes_raw.columns = ['%s%s' % (a, '%s' % b if b is not None else '') for a, b in df_price_changes_raw.columns]

# rename the columns
df_price_changes_raw.rename(columns = {'daymin':'start_day', 'daymax':'end_day'}, inplace=True)

# calculate the end of the previous period (to use to merge the current price/date range to the previous price)
df_price_changes_raw['_prev_end_day'] = df_price_changes_raw['start_day'] - 1

# merge the current price range data to the previous price
df_price_changes = pd.merge(df_price_changes_raw, df_price_changes_raw, how='left', left_on=['state_id','store_id','cat_id','dept_id','item_id','_prev_end_day'], right_on=['state_id','store_id','cat_id','dept_id','item_id','end_day'], suffixes=('','_prev'))

# drop the extra previous price range data that was merged
df_price_changes.drop(['_prev_end_day','start_day_prev','end_day_prev','_prev_end_day_prev'],axis=1,inplace=True)

# sort the data by the category ids and start day
df_price_changes.sort_values(['state_id','store_id','cat_id','dept_id','item_id','start_day'], inplace=True)

# create an indicator for whether the price change was an increase (1) or not (0)
df_price_changes['price_increase_ind'] = np.where(df_price_changes['price']>df_price_changes['price_prev'], 1, 0)

df_price_changes.head()

state_id  store_id  cat_id  dept_id  item_id  price  start_day  end_day  \
0         6         0       0        0        1   7.88          1      252   
1         6         0       0        0        1   8.88        253     1365   
2         6         0       0        0        1   9.48       1366     1969   
3         6         0       0        0        2   2.94          1      203   
4         6         0       0        0        2   3.28        204     1295   

   price_prev  price_increase_ind  
0         NaN                   0  
1        7.88                   1  
2        8.88                   1  
3         NaN                   0  
4        2.94                   1

### (7) Competition Financial Dataset (Walmart, Amazon, Target, Costco)

In [25]:
## (7) COMPETITION FINANCIAL DATASET

#Transpose dataset
wmt_df = wmt_df.T; amz_df = amz_df.T; tgt_df = tgt_df.T; cos_df = cos_df.T

# Reset Index
wmt_df.reset_index(inplace=True); amz_df.reset_index(inplace=True); tgt_df.reset_index(inplace=True); cos_df.reset_index(inplace=True)

# Rename Columns
wmt_df = wmt_df.rename(columns={'index':'Date'}); amz_df = amz_df.rename(columns={'index':'Date'}); tgt_df = tgt_df.rename(columns={'index':'Date'}); cos_df = cos_df.rename(columns={'index':'Date'})

# Drop Columns and NA values
wmt_df = wmt_df[:-1]; amz_df = amz_df[:-1]; tgt_df = tgt_df[:-2]; cos_df = cos_df[:-1]
wmt_df = wmt_df.dropna(axis=1, how='any'); amz_df = amz_df.dropna(axis=1, how='any'); cos_df = cos_df.dropna(axis=1, how='any'); tgt_df = tgt_df.dropna(axis=1, how='any')

# Format column and select columns
wmt_df['Date'] = pd.to_datetime(wmt_df['Date'],format='%Y-%m').dt.strftime('%Y'); amz_df['Date'] = pd.to_datetime(amz_df['Date'],format='%Y-%m').dt.strftime('%Y')
tgt_df['Date'] = pd.to_datetime(tgt_df['Date'],format='%Y-%m').dt.strftime('%Y'); cos_df['Date'] = pd.to_datetime(cos_df['Date'],format='%Y-%m').dt.strftime('%Y')
wmt_df = wmt_df[['Date', 'Revenue USD Mil','Operating Income USD Mil', 'Net Income USD Mil', 'Shares Mil', 'Earnings Per Share USD']]
amz_df = amz_df[['Date', 'Revenue USD Mil','Operating Income USD Mil', 'Net Income USD Mil', 'Shares Mil', 'Earnings Per Share USD']]
tgt_df = tgt_df[['Date', 'Revenue USD Mil','Operating Income USD Mil', 'Net Income USD Mil', 'Shares Mil', 'Earnings Per Share USD']]
cos_df = cos_df[['Date', 'Revenue USD Mil','Operating Income USD Mil', 'Net Income USD Mil', 'Shares Mil', 'Earnings Per Share USD']]

In [26]:
## (6) COMPETITION FINANCIAL DATASET

# Merge on date
merge1 = pd.merge(wmt_df, amz_df, on=['Date'])

# Rename columns
merge1.rename(columns={'Revenue USD Mil_x':'Walmart Revenue USD Mil',
    'Operating Income USD Mil_x':'Walmart Operating Income USD Mil',
    'Net Income USD Mil_x':'Walmart Net Income USD Mil',
    'Shares Mil_x':'Walmart Shares Mil',
    'Earnings Per Share USD_x':'Walmart Earnings Per Share USD',
    'Revenue USD Mil_y':'Amazon Revenue USD Mil',
    'Operating Income USD Mil_y':'Amazon Operating Income USD Mil',
    'Net Income USD Mil_y':'Amazon Net Income USD Mil',
    'Shares Mil_y':'Amazon Shares Mil',
    'Earnings Per Share USD_y':'Amazon Earnings Per Share USD'}, inplace=True)

# Merge on date
merge2 = pd.merge(merge1, tgt_df, on=['Date'], how='outer')

# Rename columns
merge2.rename(columns={'Revenue USD Mil': 'Target Revenue USD Mil',
    'Operating Income USD Mil':'Target Operating Income USD Mil',
    'Net Income USD Mil':'Target Net Income USD Mil',
    'Shares Mil':'Target Shares Mil',
    'Earnings Per Share USD':'Target Earnings Per Share USD'}, inplace=True)

# Merge on date
merge_df = pd.merge(merge2, cos_df, on=['Date'])

# Rename columns
merge_df.rename(columns={'Date':"date",
    'Revenue USD Mil': 'Costco Revenue USD Mil',
    'Operating Income USD Mil':'Costco Operating Income USD Mil',
    'Net Income USD Mil':'Costco Net Income USD Mil',
    'Shares Mil':'Costco Shares Mil',
    'Earnings Per Share USD':'Costco Earnings Per Share USD'}, inplace=True)

In [27]:
## (6) COMPETITION FINANCIAL DATASET CONT.

# Create tables from selected columns
revenue_df = merge_df[['date','Walmart Revenue USD Mil','Amazon Revenue USD Mil','Target Revenue USD Mil','Costco Revenue USD Mil']]
netincome_df = merge_df[['date', 'Walmart Net Income USD Mil', 'Amazon Net Income USD Mil', 'Target Net Income USD Mil', 'Costco Net Income USD Mil']]
earnings_df = merge_df[['date', 'Walmart Earnings Per Share USD', 'Amazon Earnings Per Share USD', 'Target Earnings Per Share USD', 'Costco Earnings Per Share USD']]

# Set index
revenue_df = revenue_df.set_index('date'); netincome_df = netincome_df.set_index('date'); earnings_df = earnings_df.set_index('date')